In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the libraries. 

In [ ]:
!pip install chart_studio

     |████████████████████████████████| 71kB 4.9MB/s 


In [ ]:
#importing all libraries
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import os
import re
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go
import itertools
from scipy import stats
from ast import literal_eval
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
!pip install nltk
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.stem.wordnet import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import gensim
import json

import csv
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
import keras.utils
from keras import utils as np_utils

### Functions for Preprocessing

In [ ]:
import string
import nltk
import spacy

#Input a document and receive a list of lists detailing individual word tokens.
#Note, this is a Whitespace tokenizer, which preserves contractions.
def tokenize(doc):
  return nltk.tokenize.WhitespaceTokenizer().tokenize(doc)



#Input a document and receive a list of lists detailing individual sentence tokens.
def sentence_tokenize(doc):
  return nltk.tokenize.sent_tokenize(doc)



#Input a string or document and output a list of tokens that has split contractions.
#Note, this is a WordPunct tokenizer, which splits apart contractions. However, the stopwords included in 
#NLTK should delete the split contractions if converted to lowercase.
def split_contractions(doc):
  return nltk.tokenize.wordpunct_tokenize(doc)



#Input a list of list containing tokenized document or corpus. Return all tokens if not of the format: [token] or <token>
#This is an attempt to remove semantic information that would skew training.
#NOTE: this should be done before removing punctuation, otherwise you will not find the correct tokens.
def remove_semantics(tokens):
    #first remove semantic information surrounded by brackets: []
    tokens =  [token for token in tokens if not token.startswith('[') and  not token.endswith(']')]
    #then remove semantic information surround by: <>
    return [token for token in tokens if not token.startswith('<') and  not token.endswith('>')]



#Input a tokenized text as a list of lists. 
#POS tag, and then return a list containing all tokens that are not proper nouns.
def remove_proper_nouns(tokens):
	pos = nltk.pos_tag(tokens)
	to_remove = ['NNP', 'NNPS']

	return [pos[0] for pos in pos if pos[1] not in to_remove]



#Input a list of tokens and output a single string
def list_to_string(tokens):
    string_ = ""
    whitespace = " "
    for token in tokens:
        string_ = string_ + token + whitespace
        
    return string_



#Convert all tokens to lowercase. Assumes that tokenizing has already occurred.
def lower_case(tokens):
     
    return [token.lower() for token in tokens]



#This method removes NLTK stopwords. Currently, this method works for English stopwords.
#More work can be done to input a "language" parameter to make it more customizable by language.
def remove_stopwords(tokens):
    #Create stopwords set for English.
    #This list can be modified in the english.txt file for NLTK, or use this syntax:
    #stopwords.extend(your_list_here)
    stopwords = nltk.corpus.stopwords.words('english')
    
    #add bad tokens into stopwords list
    stopwords.extend(['mmm', 'k', 'xxx'])

    #update stop words to keep certain words
    keep_list = ['uh', 'uhhh', 'um', 'ummm', 'i', "i'm", 'me', 'myself', 'mine', 'my',
                 'we', 'us', 'ourselves', 'ourself', 'ours', 'our', 'he', 'him', 'himself', 'his','she', 'her', 'herself', 'hers', 'her', 'they', 'them', 'themselves', 'themself', 'theirs', 'their', 			'always', 'nothing', 'completely', 'never', 'all', 'every', 'none', 'only']

    stopwords = [word for word in stopwords if word not in keep_list]
    stops = set(stopwords)

    return [token for token in tokens if token not in stops]



#Remove punctuation and replace it with an empty string character.
#Example: can't --> cant
def strip_punctuation(tokens):
    #Create punctuation list: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    punctuation = string.punctuation

    #See docs: str.maketrans() will make a table to pass to str.translate()
    table = str.maketrans('', '', string.punctuation)

    return [token.translate(table) for token in tokens]



#Strip any non-alphabetic characters.
def remove_numbers(tokens):
    return [token for token in tokens if token.isalpha()]



#Input a list of tokens, output a list of tokens minus people and places.
#NOTE: SpaCy's entity recognition should be done before converting tokens to lowercase, and at this time
#does not recognize lowercase entities. So it will recognize United States, but not UNITED STATES or united states. Likewise, it may recognize Sarah and SARAH, but not sarah.
def remove_people_places(doc):
    nlp = spacy.load('en')
    
    document = list_to_string(doc)
    document = nlp(document)
    
    #create list of removable words
    ppl_plcs = ['PERSON', 'GPE']
    to_remove = [ent.text for ent in document.ents if ent.label_ in ppl_plcs]
    
    #now remove those words and 
    return [token for token in doc if token not in to_remove]



#Input a file that has text on separate lines (separated by breakline characters or otherwise).
#Output number of lines in that file.
#def count_sentences(filename):
    #sentences = 0
    #reader = csv.reader(filename,delimiter = ",")
    #data = list(reader)
    #sentences = len(data)

    #return sentences

  

#Input a custom list of words you would like to count in a list of tokenized words.
#Outputs the total count of the words in your_list based on the frequency in tokens.
def list_in_text(your_list, tokens):
    count = 0

    for word in your_list:
        for token in tokens:
            if token == word:
                count += 1

    return count

### Feature Extraction for the time taken by each participant to talk

In [ ]:
#append participant number from filename for indexing
participant_num = []

#count total time per transcript
total_time = []

In [ ]:
#change directory
change_text_dir = '/content/drive/My Drive/transcripts/'
change_dir = os.chdir(change_text_dir)

#get current working directory
this_dir = os.getcwd()
filenames = os.listdir(this_dir)

#sort filenames for sorting vectors
filenames = sorted(filenames)

In [ ]:
for filename in filenames:
    if filename.endswith('.csv'):
        
        #add participant id number to list
        participant_num.append(int(filename[:3]))
            
        #read in tab-delimited files
        df = pd.read_csv(filename, delimiter='\t')
      
        #drop unnecessary columns
        df.drop(['speaker', 'value'], axis=1, inplace=True)

        #create time difference column
        df['time_per_sent'] = df.stop_time - df.start_time

        #get total transcript time in minutes
        #rounded to five decimal places
        time = np.round((df.time_per_sent.sum() / 60.0), 5)

        total_time.append(time)

In [ ]:
#create dataframe of transcript times for EDA later on
transcript_features = pd.DataFrame({'id': participant_num,
                                 'total_time': total_time})

In [ ]:
transcript_features.head()

,id,total_time
0,300,4.94333
1,301,9.55650
2,302,5.37210
3,303,13.18600
4,304,8.77833


In [ ]:
transcript_time = transcript_features.to_csv('/content/drive/My Drive/time.csv')

# Feature extraction for sentence count and average words spoken

In [ ]:
#lists to append each transcript feature to
#append participant number from filename for indexing
participant_num = []
#count total words, words per sentence
total_words, wps = [], []
#filler words, personal prnouns lists
pronouns1, pronouns2, pronouns3 = [], [], []

fillers, absolute_words = [], []

#create lists containing fillers, pronouns
filler_list = ['uh', 'uhhh', 'um', 'ummm','ugh' ]

#source: https://en.wikipedia.org/wiki/English_personal_pronouns
singular_list = ['i', "i'm", 'me', 'myself', 'mine', 'my','im']
plural_list = ['we', 'us', 'ourselves', 'ourself', 'ours', 'our']
third_list = ['he', 'him', 'himself', 'his', 'she', 'her', 'herself', 'hers', 'her',
             'they', 'them', 'themselves', 'themself', 'theirs', 'their']
laughter_list = ['laughter']

#create list for absolutist words
absolute_list = ['always', 'nothing', 'completely', 'never', 'all', 'every', 'none', 'only']

In [ ]:
sentence_total = []

In [ ]:
#create a list of *strings*, each one containing the transcript of a participant
for filename in filenames:
    if filename.endswith('.csv'):
        #add participant id number to list
        participant_num.append(int(filename[:3]))
        #read in tab-delimited files
        df = pd.read_csv(filename, delimiter='\t')
        #drop unnecessary columns
        df.drop(['speaker'], axis=1, inplace=True)
        sentence_count = df.value.count()
       
        #whitespace tokenize
        tokens = df.value.apply(lambda x : tokenize(str(x)))
        total = len(tokens)
        total_words.append(total)

        average_words = total / sentence_count
        

        
    sentence_total.append(sentence_count)
    wps.append(average_words)

In [ ]:
sentence_total.pop()
wps.pop()

1.0

In [ ]:
len(sentence_total)


189

In [ ]:
transcript_features['sentence_count'] = sentence_total
transcript_features['wps'] = wps

In [ ]:
transcript_features.head()

,id,total_time,sentence_count,wps
0,300,4.94333,295,1.0
1,301,9.55650,174,1.0
2,302,5.37210,181,1.0
3,303,13.18600,186,1.0
4,304,8.77833,191,1.0


### Feature extraction for counting the filler words 

In [ ]:
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            person_id = m.group(1)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            for i in range(0, lines):
                row = transcript.iloc[i]
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    if question != "":
                        rows_list.append(p)
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all.csv', sep=',')
    print("File was created")
    return all_participants

In [ ]:
#loading the data
data_path = "/content/drive/My Drive/transcripts/"
all_participants = transcripts_to_dataframe(data_path)

File was created


In [ ]:
all_participants['tokenized_answer'] = all_participants.apply(lambda row: nltk.word_tokenize(row['answer']), axis=1)

In [ ]:
all_participants.head()

,personId,question,answer,tokenized_answer
0,300,hi i'm ellie thanks for coming in today,,[]
1,300,i was created to talk to people in a safe and ...,,[]
2,300,think of me as a friend i don't judge i can't ...,,[]
3,300,i'm here to learn about people and would love ...,,[]
4,300,i'll ask a few questions to get us started and...,,[]


In [ ]:
#Selecting the data from the data from based on the questions asked  ['where', 'when', 'how', 'why', 'are', 'what', 'do', 'have', 'can', 'did', 'is', 'could', 'so', 'tell', 'who', 'has']
interrogative = ["where", "when", "how","why","are","what","do","have","can","did","is", "could", "so", "tell", "who", "has"]
rslt_df = all_participants[all_participants.question.str.contains('|'.join(interrogative),na=False)]
rslt_df['answer'].replace('', np.nan, inplace=True)
rslt_df.dropna(subset = ["answer"], inplace=True)
rslt_df.reset_index(drop=True, inplace=True)
rslt_df
#rslt_df = rslt_df.to_csv("/content/drive/My Drive/rslt_df.csv")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,personId,question,answer,tokenized_answer
0,300,how are you doing today,good,[good]
1,300,where are you from originally,atlanta georgia,"[atlanta, georgia]"
2,300,why'd you move to l_a,um my parents are from here um,"[um, my, parents, are, from, here, um]"
3,300,how do you like l_a,i love it,"[i, love, it]"
4,300,what are some things you really like about l_a,i like the weather i like the opportunities u...,"[i, like, the, weather, i, like, the, opportun..."
...,...,...,...,...
8698,390,that's so good to hear,mm,[mm]
8699,390,is there anything you regret,um hm no um except meeting that one woman uh,"[um, hm, no, um, except, meeting, that, one, w..."
8700,390,what advice would you give to yourself ten or ...,uh i don't know probably try a little harder ...,"[uh, i, do, n't, know, probably, try, a, littl..."
8701,390,tell me how you spend your ideal weekend,oh um getting out of town um going going away...,"[oh, um, getting, out, of, town, um, going, go..."


In [ ]:
rslt_df.sample(10)

,personId,question,answer,tokenized_answer
7410,465,that sounds really hard,it was cool it was fun life,"[it, was, cool, it, was, fun, life]"
6721,452,tell me about the last time you felt really happy,uh right now i feel really happy,"[uh, right, now, i, feel, really, happy]"
7953,478,what got you to seek help,uh i was i was just um very sad very depresse...,"[uh, i, was, i, was, just, um, very, sad, very..."
564,316,is going to a therapist helping you,to a degree but not as much as i like,"[to, a, degree, but, not, as, much, as, i, like]"
7802,472,tell me about your kids,um my daughter is um thirty nine beautiful br...,"[um, my, daughter, is, um, thirty, nine, beaut..."
441,303,can you give me an example of that,well um by you being the first teacher your k...,"[well, um, by, you, being, the, first, teacher..."
3145,373,what made you decide to do that,ugh just a abundance of <laughter> reasons th...,"[ugh, just, a, abundance, of, <, laughter, >, ..."
6097,433,like what,uh like building my car differently um gettin...,"[uh, like, building, my, car, differently, um,..."
8250,364,how old were you when you joined the military,that um seventeen i started when after high s...,"[that, um, seventeen, i, started, when, after,..."
1272,326,what do you think of today's kids,huh,[huh]


In [ ]:
rslt_df['word_count'] = rslt_df['answer'].apply(lambda x: len(str(x).split(" "))-1)
rslt_df[['answer','word_count']].head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,answer,word_count
0,good,1
1,atlanta georgia,2
2,um my parents are from here um,7
3,i love it,3
4,i like the weather i like the opportunities u...,10


In [ ]:
rslt_df['fill'] = rslt_df.tokenized_answer.apply(lambda x : list_in_text(filler_list, x))
rslt_df['single'] = rslt_df.tokenized_answer.apply(lambda x : list_in_text(singular_list, x))
rslt_df['plural'] = rslt_df.tokenized_answer.apply(lambda x : list_in_text(plural_list, x))
rslt_df['third'] = rslt_df.tokenized_answer.apply(lambda x : list_in_text(third_list, x))
rslt_df['absolute'] = rslt_df.tokenized_answer.apply(lambda x : list_in_text(absolute_list, x))
rslt_df['laughter_count'] = rslt_df.tokenized_answer.apply(lambda x : list_in_text(laughter_list, x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
rslt_df['laughter_count'].sample(10)

1741    0
2353    0
3234    0
7037    1
7609    0
523     0
4492    0
4093    0
6176    1
3268    0
Name: laughter_count, dtype: int64

In [ ]:
rslt_df.sample(10)

,personId,question,answer,tokenized_answer,word_count,fill,single,plural,third,absolute,laughter_count
1737,336,have you noticed any changes in your behavior ...,behavior what,"[behavior, what]",2,0,0,0,0,0,0
37,300,what are you like when you don't sleep well,irritated um lazy,"[irritated, um, lazy]",3,1,0,0,0,0,0
8456,399,why,i don't know it's just always been like that ...,"[i, do, n't, know, it, 's, just, always, been,...",24,0,1,0,1,2,0
5129,491,that sounds like a great situation,and i it just i moved past the fear and just ...,"[and, i, it, just, i, moved, past, the, fear, ...",15,0,3,0,0,0,0
5238,416,can you tell me about that,um we're relationship just wasn't doing very ...,"[um, we, 're, relationship, just, was, n't, do...",79,1,3,2,2,2,0
5064,489,why,because that makes you have attention deficit...,"[because, that, makes, you, have, attention, d...",23,0,0,0,0,0,0
2234,347,what do you do now,uh certified nurse's aide,"[uh, certified, nurse, 's, aide]",4,1,0,0,0,0,0
7483,463,what's something you feel guilty about,um this one time i <sigh> at my work uh it's ...,"[um, this, one, time, i, <, sigh, >, at, my, w...",23,2,4,0,0,0,0
2467,351,yeah i'm sorry to hear that,it's okay <laughter>,"[it, 's, okay, <, laughter, >]",3,0,0,0,0,0,1
1956,339,are you okay with this,yes,[yes],1,0,0,0,0,0,0


##Counting the Positive and Negative Words used by participants

In [ ]:
#change directory
with_i_text_dir = '/content/drive/My Drive/transcripts/'

#positive and negative word lists obtained from here:
#http://www.wjh.harvard.edu/~inquirer/homecat.htm
pos_neg_file = '/content/drive/My Drive/inquirerbasic.xls'

In [ ]:
df = pd.read_excel(pos_neg_file)

df.head()

,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst,Academ,Doctrin,Econ@,Exch,ECON,Exprsv,Legal,Milit,Polit@,POLIT,Relig,Role,COLL,Work,Ritual,SocRel,Race,...,AffOth,AffTot,WltPt,WltTran,WltOth,WltTot,WlbGain,WlbLoss,WlbPhys,WlbPsyc,WlbPt,WlbTot,EnlGain,EnlLoss,EnlEnds,EnlPt,EnlOth,EnlTot,SklAsth,SklPt,SklOth,SklTot,TrnGain,TrnLoss,TranLw,MeansLw,EndsLw,ArenaLw,PtLw,Nation,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
0,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
1,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,Weak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,AffTot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
2,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,Weak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
3,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Passive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TranLw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
4,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,NaN


In [ ]:
print(df.shape)

(11788, 186)


In [ ]:
#right now we have all word lists, we need to isolate just the positive and negative words
df = df[['Entry', 'Positiv', 'Negativ', 'PosAff', 'NegAff']]

df.head()

,Entry,Positiv,Negativ,PosAff,NegAff
0,A,NaN,NaN,NaN,NaN
1,ABANDON,NaN,Negativ,NaN,NaN
2,ABANDONMENT,NaN,Negativ,NaN,NaN
3,ABATE,NaN,Negativ,NaN,NaN
4,ABATEMENT,NaN,NaN,NaN,NaN


In [ ]:
#we need to get rid of nan values, only take explicitly labeled data
positive = df['Entry'][df['Positiv']=='Positiv']
positive.dropna(inplace=True)

negative = df['Entry'][df['Negativ']=='Negativ']
negative.dropna(inplace=True)

pos_aff = df['Entry'][df['PosAff']=='PosAff']
pos_aff.dropna(inplace=True)

neg_aff = df['Entry'][df['NegAff']=='NegAff']
neg_aff.dropna(inplace=True)

#we should also convert each word list to lowercase
#and the pandas series object to a list
positive = positive.str.lower().tolist()
negative = negative.str.lower().tolist()
pos_aff = pos_aff.str.lower().tolist()
neg_aff = neg_aff.str.lower().tolist()

In [ ]:
#according to the website, there are 1915 pos, 2291 neg
len_pos = len(positive)
len_neg = len(negative)

print('There are {} positive words in the positive word list.'.format(len_pos))
print('There are {} negative words in the negative word list.'.format(len_neg), '\n')

#and there should be 126 pos_aff, 193 neg_aff words
print('There are {} positive words in the positive affect word list.'.format(len(pos_aff)))
print('There are {} negative words in the negative affect word list.'.format(len(neg_aff)), '\n')

There are 1915 positive words in the positive word list.
There are 2291 negative words in the negative word list. 

There are 126 positive words in the positive affect word list.
There are 193 negative words in the negative affect word list. 



In [ ]:
print(positive[:5], '\n')
print(negative[:5])


['abide', 'ability', 'able', 'abound', 'absolve'] 

['abandon', 'abandonment', 'abate', 'abdicate', 'abhor']


In [ ]:
positive.extend(pos_aff)
negative.extend(neg_aff)

#now convert to a set to remove any duplicates
positive = list(set(positive))
negative = list(set(negative))

In [ ]:
print('There are {} more positive words in the positive word list now.'.format(len(positive) - len_pos))
print('There are {} more negative words in the negative word list now.'.format(len(negative) - len_neg), '\n')

There are 24 more positive words in the positive word list now.
There are 27 more negative words in the negative word list now. 



In [ ]:
#check for common positive words
print('good' in positive)
print('great' in positive)
print('happy' in positive)
print('love' in positive)
print('excellent' in positive, '\n')

#check for common negative words
print('bad' in negative)
print('terrible' in negative)
print('sad' in negative)
print('depressed' in negative)
print('tired' in negative)
print('bored' in negative)
print('alone' in negative)

False
False
False
False
True 

True
True
True
False
False
False
False


In [ ]:
positive.append('good')
positive.append('great')
positive.append('love')
positive.append('happy')
positive.append('like')

#and now depressed, tired, bored, alone, plus some others
negative.append('depressed')
negative.append('tired')
negative.append('bored')
negative.append('alone')
negative.append('annoying')
negative.append('irritate')
negative.append('irritated')
negative.append('bother')
negative.append('bothered')

In [ ]:
#count positive words unions
rslt_df['pos_in_tokens'] =  rslt_df.tokenized_answer.apply(lambda x : list_in_text(positive, x))
rslt_df['neg_in_tokens'] =  rslt_df.tokenized_answer.apply(lambda x : list_in_text(negative, x))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
rslt_df

,personId,question,answer,tokenized_answer,word_count,fill,single,plural,third,absolute,pos_in_tokens,neg_in_tokens
0,300,how are you doing today,good,[good],2,0,0,0,0,0,1,0
1,300,where are you from originally,atlanta georgia,"[atlanta, georgia]",3,0,0,0,0,0,0,0
2,300,why'd you move to l_a,um my parents are from here um,"[um, my, parents, are, from, here, um]",8,2,1,0,0,0,0,0
3,300,how do you like l_a,i love it,"[i, love, it]",4,0,1,0,0,0,1,0
4,300,what are some things you really like about l_a,i like the weather i like the opportunities u...,"[i, like, the, weather, i, like, the, opportun...",11,1,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8698,390,that's so good to hear,mm,[mm],2,1,0,0,0,0,0,0
8699,390,is there anything you regret,um hm no um except meeting that one woman uh,"[um, hm, no, um, except, meeting, that, one, w...",11,3,0,0,0,0,0,0
8700,390,what advice would you give to yourself ten or ...,uh i don't know probably try a little harder ...,"[uh, i, do, n't, know, probably, try, a, littl...",66,11,4,1,0,0,1,2
8701,390,tell me how you spend your ideal weekend,oh um getting out of town um going going away...,"[oh, um, getting, out, of, town, um, going, go...",22,5,0,0,0,0,0,0
